In [1]:
import json
import yaml
import requests
import folium
from pymongo import MongoClient
import logging
logging.basicConfig(filename='log.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


import random

## Weather

In [2]:
def GetWeather(coord:list):
    """Coordinates coord= [lat, lon], where lat, lon is latitude and longitude in type float. \n
    Example: [19.29586, -97.768889]. \n
    Invalid formats: 18°55'22.8\" N 98°23'44.3\"W"""
    lat= coord[0]
    lon = coord[1]
    API_key=config['API_key']
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric'
    response = requests.get(url)
    data_weather = response.json()
    return data_weather   

In [3]:
def verification(diccionario, valor):

    for calidad, rango in diccionario.items():
        if 'min' in rango and 'max' in rango:
            if rango['min'] <= valor < rango['max']:
                return calidad
        elif 'min' in rango and valor >= rango['min']:
            return calidad

    return None

### Wind

|Qualitative name   |Index  |     wind      |
|-------------------|-------|---------------|
|Calmed down 	    |1 	    |[0;1.4) 	    |
|Soft breeze        |2 	    |[1.4; 5.4) 	|
|Moderate Breeze    |3 	    |[5.4; 10.6) 	|
|Strong wind        |4 	    |[10.6; 17)     |
|Storm   	        |5 	    |⩾17 	        |


Unidad: $m/s$   

In [8]:
data_weather = GetWeather(config['cities']['Atlixco'])
wind_speed = data_weather['wind']['speed']
wind_gust = data_weather['wind']['gust']

In [15]:
def check_wind_speed(diccionario:dict, valor:float):
    Index= {'Calmed down':1, 'Soft breeze':2, 'Moderate Breeze':3, 'Strong wind': 4, 'Storm':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return (ver_txt, Index[ver_txt], valor, 'm/s')

In [18]:
windStandars =  config['notifications']['weather']['wind']
print(wind_speed)
check_wind_speed(windStandars,wind_speed)


0.48


('Calmed down', 1, 0.48, 'm/s')

### Temp

|Qualitative name  |Index  |     wind      |
|------------------|-------|---------------|
|Frozen	           |1 	    |[-20,0) 	    |
|Cold              |2 	    |[0,10) 	|
|Moderate          |3 	    |[10,25) 	|
|Warm              |4 	    |[25,35)     |
|Very warm         |5 	    |⩾35        |


Unidad: $°C$   


In [23]:
data_weather = GetWeather(config['cities']['Atlixco'])

temp = data_weather['main']['temp'] #°C
humidity = data_weather['main']['humidity'] # Humidity, %

In [30]:
def check_temp(diccionario:dict, valor:float):
    Index= {'Frozen':1, 'Cold':2, 'Moderate':3, 'Warm': 4, 'Very warm':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': '°C'}


In [29]:
weatherStandars =  config['notifications']['weather']['temp']
print(temp)
check_temp(weatherStandars,temp)

25.12


('Warm', 4, 25.12, '°C')